In [ ]:
!pip install pyomo==6.7.1

In [ ]:
from pathlib import Path

import pandas as pd

In [ ]:
excel_file = Path("../data/ChallengeXHEC23022024.xlsx")

In [ ]:
schedule = pd.read_excel(excel_file, sheet_name=0)
clients = pd.read_excel(excel_file, sheet_name=1)
caregivers = pd.read_excel(excel_file, sheet_name=2)

In [ ]:
schedule.Prestation.unique()

In [ ]:
discard_list = [
    "ADMINISTRATION",
    "VISITE MEDICALE",
    "FORMATION",
    "COORDINATION",
    "HOMMES TOUTES MAINS",
]

In [ ]:
sched_one_day = schedule.loc[schedule.Date == "2024-01-03", :]

In [ ]:
sched_one_day.Prestation.value_counts()

In [ ]:
sched_one_day = sched_one_day[~sched_one_day.Prestation.isin(discard_list)]

In [ ]:
sched_one_day["idx"] = sched_one_day.index

In [ ]:
from datetime import datetime

common_date = sched_one_day["Date"].iloc[0]
sched_one_day["Heure de fin"] = sched_one_day["Heure de fin"].apply(
    lambda x: datetime.combine(common_date, x)
)
sched_one_day["Heure de début"] = sched_one_day["Heure de début"].apply(
    lambda x: datetime.combine(common_date, x)
)
sched_one_day["Duration"] = (
    sched_one_day["Heure de fin"] - sched_one_day["Heure de début"]
)
sched_one_day["Duration"] = sched_one_day["Duration"].apply(
    lambda x: x.seconds // 60
)

In [ ]:
sched_one_day

In [ ]:
caregivers

In [ ]:
schedule

In [ ]:
care_clients_commute = pd.read_csv("../data/commute_driving_care_clients.csv")

In [ ]:
care_clients_commute.shape

In [ ]:
clients_care_commute = pd.read_csv("../data/commute_driving_clients_care.csv")

In [ ]:
clients_commute = pd.read_csv("../data/commute_driving_clients.csv")

In [ ]:
schedule["ID Client"].nunique()

In [ ]:
from itertools import product, combinations

len(
    list(
        product(schedule["ID Client"].unique(), schedule["ID Client"].unique())
    )
)

In [ ]:
len(
    list(
        product(
            schedule["ID Client"].unique(), schedule["ID Intervenant"].unique()
        )
    )
)

In [ ]:
len(list(combinations(schedule["ID Client"].unique(), 2)))

In [ ]:
clients_commute

In [ ]:
(clients_commute.source == clients_commute.destination).sum()

In [ ]:
list(
    clients_commute[["source", "destination", "commute_seconds"]].itertuples(
        index=False, name=None
    )
)

In [ ]:
# Import pyomo libraries
import pyomo.environ as pe
import pyomo.gdp as pyogdp

from itertools import product

In [ ]:
test = sched_one_day["Heure de début"].iloc[0] - pd.to_datetime(
    "2024-01-03 00:00:00"
)

test.seconds // 60

In [ ]:
class CareScheduler:
    def __init__(self, excel_file):
        """
        Read cargiver and session data into Pandas DataFrames
        Args:
            excel_file (str): path to data in excel format
        """
        try:
            clients = pd.read_excel(excel_file, sheet_name=1)
            sessions = pd.read_excel(excel_file, sheet_name=0)

            # NOTE: remove later
            sessions = sessions.loc[sessions.Date == "2024-01-03", :]
            sessions["idx"] = sessions.index

            # create duration
            common_date = sessions["Date"].iloc[0]
            sessions["Heure de fin"] = sessions["Heure de fin"].apply(
                lambda x: datetime.combine(common_date, x)
            )
            sessions["Heure de début"] = sessions["Heure de début"].apply(
                lambda x: datetime.combine(common_date, x)
            )
            sessions["Duration"] = (
                sessions["Heure de fin"] - sessions["Heure de début"]
            )
            sessions["Duration"] = sessions["Duration"].apply(
                lambda x: x.seconds // 60
            )

            sessions["Start_time"] = sessions[
                "Heure de début"
            ] - pd.to_datetime("2024-01-03 00:00:00")
            sessions["Start_time"] = sessions["Start_time"].apply(
                lambda x: x.seconds // 60
            )

            # discard
            discard_list = [
                "ADMINISTRATION",
                "VISITE MEDICALE",
                "FORMATION",
                "COORDINATION",
                "HOMMES TOUTES MAINS",
            ]
            sessions = sessions[~sessions.Prestation.isin(discard_list)]

            sessions = sessions.merge(clients, how="left", on="ID Client")
            sessions = sessions.drop(columns="ID Intervenant")
            self.df_sessions = sessions
        except FileNotFoundError:
            print("Session data not found.")

        try:
            self.df_cargeivers = pd.read_excel(excel_file, sheet_name=2)
            # TODO: take kaans matrix -> filter cargeivers if they are not available today
        except FileNotFoundError:
            print("Caregiver data not found")

        try:
            df_clients_commute = pd.read_csv(
                "../data/commute_driving_clients.csv"
            )
            df_clients_commute["commute_minutes"] = (
                df_clients_commute["commute_seconds"] / 60
            )
            self.df_clients_commute = df_clients_commute
        except FileNotFoundError:
            print("Commute between clients data not found")

        try:
            df_care_clients_commute = pd.read_csv(
                "../data/commute_driving_care_clients.csv"
            )
            df_care_clients_commute["commute_minutes"] = (
                df_care_clients_commute["commute_seconds"] / 60
            )
            self.df_care_clients_commute = df_care_clients_commute
        except FileNotFoundError:
            print("Commute between clients data not found")

        try:
            df_clients_care_commute = pd.read_csv(
                "../data/commute_driving_clients_care.csv"
            )
            df_clients_care_commute["commute_minutes"] = (
                df_clients_care_commute["commute_seconds"] / 60
            )
            self.df_clients_care_commute = df_clients_care_commute
        except FileNotFoundError:
            print("Commute between clients data not found")

        self.model = self.create_model()

    def _generate_case_durations(self):
        """
        Generate mapping of cases IDs to case time for the procedure
        Returns:
            (dict): dictionary with CaseID as key and median case time (mins) for procedure as value
        """
        return pd.Series(
            self.df_sessions["Duration"].values, index=self.df_sessions["idx"]
        ).to_dict()

    def _generate_start_time(self, tasks):
        tasks_df = pd.DataFrame(tasks, columns=["idx", "Cargegiver_ID"])

        temp = pd.DataFrame(
            self.df_sessions["Start_time"].values,
            index=self.df_sessions["idx"],
            columns=["Start_time"],
        )  # .to_dict()

        temp = tasks_df.merge(
            temp, how="left", right_index=True, left_on="idx"
        )

        arrays = [
            temp["idx"].to_list(),
            temp["Cargegiver_ID"].to_list(),
        ]
        tuples = list(zip(*arrays))

        index = pd.MultiIndex.from_tuples(
            tuples, names=["idx", "Cargegiver_ID"]
        )
        temp = pd.Series(temp["Start_time"].values, index=index)

        return temp.to_dict()

    def _generate_clients_commute(self):
        clients_commute = {}
        for source, dest in product(
            self.df_sessions["ID Client"].unique(),
            self.df_sessions["ID Client"].unique(),
        ):
            clients_commute[(source, dest)] = self.df_clients_commute.loc[
                (self.df_clients_commute.source == source)
                & (self.df_clients_commute.destination == dest),
                "commute_minutes",
            ].iloc[0]
        return clients_commute

    def _generate_care_clients_commute(self):
        care_clients_commute = {}
        for source, dest in product(
            self.df_cargeivers["ID Intervenant"].unique(),
            self.df_sessions["ID Client"].unique(),
        ):
            care_clients_commute[
                (source, dest)
            ] = self.df_care_clients_commute.loc[
                (self.df_care_clients_commute.source == source)
                & (self.df_care_clients_commute.destination == dest),
                "commute_minutes",
            ].iloc[
                0
            ]
        return care_clients_commute

    def _generate_clients_care_commute(self):
        clients_care_commute = {}
        for source, dest in product(
            self.df_sessions["ID Client"].unique(),
            self.df_cargeivers["ID Intervenant"].unique(),
        ):
            clients_care_commute[
                (source, dest)
            ] = self.df_clients_care_commute.loc[
                (self.df_clients_care_commute.source == source)
                & (self.df_clients_care_commute.destination == dest),
                "commute_minutes",
            ].iloc[
                0
            ]
        return clients_care_commute

    def _IDX_CLIENTS_match(self):
        return pd.Series(
            self.df_sessions["ID Client"].values, index=self.df_sessions["idx"]
        ).to_dict()

    def _generate_case_bigger(self):
        case_bigger = {}
        for case1, case2 in product(
            self.df_sessions["idx"].unique(), self.df_sessions["idx"].unique()
        ):
            case_bigger[(case1, case2)] = int(case1 > case2)
        return case_bigger

    def _generate_disjunctions(self):
        """Returns:
        disjunctions (list): list of tuples containing disjunctions
        """
        cases = self.df_sessions["idx"].to_list()
        cargivers = self.df_cargeivers["ID Intervenant"].to_list()
        disjunctions = []
        for (case1, case2, cargiver) in product(cases, cases, cargivers):
            if (case1 != case2) and (
                case2,
                case1,
                cargiver,
            ) not in disjunctions:
                disjunctions.append((case1, case2, cargiver))

        return disjunctions

    def create_model(self):
        model = pe.ConcreteModel()

        # List of case IDs in home care client needs list
        model.CASES = pe.Set(initialize=self.df_sessions["idx"].tolist())
        # List of potential caregiver IDs
        model.CAREGIVERS = pe.Set(
            initialize=self.df_cargeivers["ID Intervenant"].tolist()
        )
        # List of tasks - all possible (caseID, caregiverID) combination
        model.TASKS = pe.Set(
            initialize=model.CASES * model.CAREGIVERS, dimen=2
        )

        # The duration (expected case time) for each operation
        model.CASE_DURATION = pe.Param(
            model.CASES, initialize=self._generate_case_durations()
        )
        # Start time of a case
        model.CASE_START_TIME = pe.Param(
            model.TASKS, initialize=self._generate_start_time(model.TASKS)
        )

        model.CLIENT_CONNECTIONS = pe.Set(
            initialize=product(
                self.df_sessions["ID Client"].unique(),
                self.df_sessions["ID Client"].unique(),
            )
        )
        model.CLIENT_CARE = pe.Set(
            initialize=product(
                self.df_sessions["ID Client"].unique(),
                self.df_cargeivers["ID Intervenant"].unique(),
            )
        )
        model.CARE_CLIENT = pe.Set(
            initialize=product(
                self.df_cargeivers["ID Intervenant"].unique(),
                self.df_sessions["ID Client"].unique(),
            )
        )
        model.IDX_CLIENTS = pe.Param(
            model.CASES, initialize=self._IDX_CLIENTS_match()
        )
        model.COMMUTE = pe.Param(
            model.CLIENT_CONNECTIONS,
            initialize=self._generate_clients_commute(),
        )
        model.COMMUTE_CARE_CLIENT = pe.Param(
            model.CARE_CLIENT,
            initialize=self._generate_care_clients_commute(),
        )
        model.COMMUTE_CLIENT_CARE = pe.Param(
            model.CLIENT_CARE,
            initialize=self._generate_clients_care_commute(),
        )

        # Decision Variables
        ub = 1440  # minutes in a day
        model.M = pe.Param(initialize=1e3 * ub)  # big M
        # max_util = 0.85

        # Binary flag, 1 if case is assigned to session, 0 otherwise
        model.SESSION_ASSIGNED = pe.Var(model.TASKS, domain=pe.Binary)
        # Downtime of a caregiver caregiver
        model.DOWN_TIME = pe.Var(
            model.CAREGIVERS, bounds=(0, ub), within=pe.PositiveReals
        )
        # commute for cargiver
        model.COMMUTE_CARE = pe.Var(
            model.CAREGIVERS, bounds=(0.0, 1440.0), within=pe.PositiveReals
        )
        # Session utilisation
        model.DISJUNCTIONS = pe.Set(
            initialize=self._generate_disjunctions(), dimen=3
        )

        # Objective
        def objective_function(model):
            return pe.summation(model.COMMUTE_CARE) + 0.01 * pe.summation(
                model.DOWN_TIME
            )

        model.OBJECTIVE = pe.Objective(
            rule=objective_function, sense=pe.minimize
        )

        # Constraints
        # Cases can be assigned to a maximum of one caregiver
        def session_assignment(model, case):
            return (
                sum(
                    [
                        model.SESSION_ASSIGNED[(case, caregiver)]
                        for caregiver in model.CAREGIVERS
                    ]
                )
                == 1
            )

        model.SESSION_ASSIGNMENT = pe.Constraint(
            model.CASES, rule=session_assignment
        )

        # Constraint 6: Utilisation is defined as fraction of a theatre session taken up by cases
        def theatre_util(model, session):
            return model.DOWN_TIME[session] == 1440 - sum(
                [
                    model.SESSION_ASSIGNED[case, session]
                    * model.CASE_DURATION[case]
                    for case in model.CASES
                ]
            )

        model.THEATRE_UTIL = pe.Constraint(model.CAREGIVERS, rule=theatre_util)

        model.CASE_COMBINATIONS = pe.Set(
            initialize=product(
                self.df_sessions["idx"].unique(),
                self.df_sessions["idx"].unique(),
            ),
            dimen=2,
        )

        model.CASE_BIGGER = pe.Param(
            model.CASE_COMBINATIONS, initialize=self._generate_case_bigger()
        )

        def commute_care(model, caregiver):
            commute_expr = sum(
                [
                    model.SESSION_ASSIGNED[case[0], caregiver]
                    * model.SESSION_ASSIGNED[case[1], caregiver]
                    * (
                        model.CASE_BIGGER[case[1], case[0]]
                        * model.COMMUTE[
                            (
                                model.IDX_CLIENTS[case[0]],
                                model.IDX_CLIENTS[case[1]],
                            )
                        ]
                        + (1 - model.CASE_BIGGER[case[1], case[0]])
                        * model.COMMUTE[
                            (
                                model.IDX_CLIENTS[case[1]],
                                model.IDX_CLIENTS[case[0]],
                            )
                        ]
                    )
                    for case in model.CASE_COMBINATIONS
                ]
            )
            return model.COMMUTE_CARE[caregiver] == commute_expr

        # TODO: do it with tasks instead of per caregiver
        # def commute_care(model, caregiver):
        #     commute_expr = sum(
        #         [
        #             model.SESSION_ASSIGNED[case, caregiver]
        #             * (
        #                 model.COMMUTE_CARE_CLIENT[
        #                     (caregiver, model.IDX_CLIENTS[case])
        #                 ]
        #                 + model.COMMUTE_CLIENT_CARE[
        #                     (model.IDX_CLIENTS[case], caregiver)
        #                 ]
        #             )
        #             # * (model.smallest[case1])
        #             # * (model.largest[case1])
        #             for case in model.CASES
        #         ]
        #     )
        #     return model.COMMUTE_CARE[caregiver] == commute_expr

        model.COMMUTE_CARE_CONST = pe.Constraint(
            model.CAREGIVERS, rule=commute_care
        )

        # Constraint 6: Make sure the caregivers have shifts of 9 hours
        # somehow this does not work because of non constant pyomo expressions -> also does not look like it in the original schedule
        # def working_hours(model, caregiver):
        #     return 540 >= max([model.SESSION_ASSIGNED[case, caregiver]*model.CASE_START_TIME[case, caregiver] for case in model.CASES]) - min([model.SESSION_ASSIGNED[case, caregiver]*model.CASE_START_TIME[case, caregiver] for case in model.CASES])
        # model.WORKING_HOURS = pe.Constraint(model.CAREGIVERS, rule=working_hours)

        # TODO: -> question 3 if someone has no work
        # def compare_utils(model, caregiver):
        #     return model.UTILISATION[caregiver] >= 0.13
        # model.ALL_MIN = pe.Constraint(model.CAREGIVERS, rule=compare_utils)

        # Constraint 5: No two cases can overlap
        # def no_case_overlap(model, case1, case2, caregiver):
        #     return [
        #         model.CASE_START_TIME[case1, caregiver]
        #         + model.CASE_DURATION[case1]
        #         + model.COMMUTE[
        #             (model.IDX_CLIENTS[case1], model.IDX_CLIENTS[case2])
        #         ]
        #         <= model.CASE_START_TIME[case2, caregiver]
        #         + (
        #             (
        #                 2
        #                 - model.SESSION_ASSIGNED[case1, caregiver]
        #                 - model.SESSION_ASSIGNED[case2, caregiver]
        #             )
        #             * model.M
        #         ),
        #         model.CASE_START_TIME[case2, caregiver]
        #         + model.CASE_DURATION[case2]
        #         + model.COMMUTE[
        #             (model.IDX_CLIENTS[case2], model.IDX_CLIENTS[case1])
        #         ]
        #         <= model.CASE_START_TIME[case1, caregiver]
        #         + (
        #             (
        #                 2
        #                 - model.SESSION_ASSIGNED[case1, caregiver]
        #                 - model.SESSION_ASSIGNED[case2, caregiver]
        #             )
        #             * model.M
        #         ),
        #     ]
        def no_case_overlap(model, case1, case2, caregiver):
            return [
                model.CASE_START_TIME[case1, caregiver]
                + model.CASE_DURATION[case1]
                + model.COMMUTE_CLIENT_CARE[
                    (model.IDX_CLIENTS[case1], caregiver)
                ]
                + model.COMMUTE_CARE_CLIENT[
                    (caregiver, model.IDX_CLIENTS[case2])
                ]
                <= model.CASE_START_TIME[case2, caregiver]
                + (
                    (
                        2
                        - model.SESSION_ASSIGNED[case1, caregiver]
                        - model.SESSION_ASSIGNED[case2, caregiver]
                    )
                    * model.M
                ),
                model.CASE_START_TIME[case2, caregiver]
                + model.CASE_DURATION[case2]
                + model.COMMUTE_CLIENT_CARE[
                    (model.IDX_CLIENTS[case2], caregiver)
                ]
                + model.COMMUTE_CARE_CLIENT[
                    (caregiver, model.IDX_CLIENTS[case1])
                ]
                <= model.CASE_START_TIME[case1, caregiver]
                + (
                    (
                        2
                        - model.SESSION_ASSIGNED[case1, caregiver]
                        - model.SESSION_ASSIGNED[case2, caregiver]
                    )
                    * model.M
                ),
            ]

        model.DISJUNCTIONS_RULE = pyogdp.Disjunction(
            model.DISJUNCTIONS, rule=no_case_overlap
        )

        # pe.TransformationFactory("gdp.bigm").apply_to(model)

        return model

    def solve(self):
        solver = pe.SolverFactory(
            "ipopt", executable="/opt/homebrew/bin/ipopt"
        )

        # Solve model (verbose)
        solver_results = solver.solve(
            self.model,
            tee=True,
            # time_limit=60,
            # # mip_solver="glpk",
            # nlp_solver="ipopt",
        )
        return solver_results

In [ ]:
from pyutilib.services import register_executable

register_executable(name="glpsol")
register_executable(name="ipopt")

In [ ]:
scheduler = CareScheduler(excel_file)

In [ ]:
solver_results = scheduler.solve()

In [ ]:
model = scheduler.model

In [ ]:
model.COMMUTE_CARE.display()

In [ ]:
model.DOWN_TIME.display()  # .extract_values()

In [ ]:
actions = [
    k for k, v in model.SESSION_ASSIGNED.extract_values().items() if v == 1
]

In [ ]:
import numpy as np

In [ ]:
sorted(set(list(model.SESSION_ASSIGNED.extract_values().values())))

In [ ]:
actions[0]

In [ ]:
actions_df = pd.DataFrame(actions, columns=["idx", "Caregiver_ID"])

In [ ]:
temp = scheduler.df_sessions.copy()
temp = temp.merge(actions_df, how="left", on="idx")
temp

In [ ]:
temp.Caregiver_ID.isna().mean()

In [ ]:
temp.style

In [ ]:
# model.pprint()
# model.x.pprint()

In [ ]:
model.OBJECTIVE.display()

In [ ]:
import plotly.express as px

In [ ]:
def plot_agenda_for_intervenant(intervenant_id):
    intervenant_agenda = temp[temp["Caregiver_ID"] == intervenant_id]
    intervenant_agenda_sorted = intervenant_agenda.sort_values(
        by=["Date", "Heure de début"]
    )

    df_timeline = intervenant_agenda_sorted.copy()
    df_timeline["Start"] = pd.to_datetime(df_timeline["Heure de début"])
    df_timeline["Finish"] = pd.to_datetime(df_timeline["Heure de fin"])
    df_timeline["Task"] = df_timeline["Prestation"]
    df_timeline["Resource"] = df_timeline["Caregiver_ID"].astype(str)

    fig = px.timeline(
        df_timeline,
        x_start="Start",
        x_end="Finish",
        y="Task",
        color="Resource",
    )
    fig.update_yaxes(autorange="reversed")
    fig.update_layout(title=f"Agenda for Intervenant ID: {intervenant_id}")

    fig.update_layout(
        xaxis=dict(
            rangeselector=dict(
                buttons=list(
                    [
                        dict(
                            count=1,
                            label="1D",
                            step="day",
                            stepmode="backward",
                        ),
                        dict(
                            count=7,
                            label="1W",
                            step="day",
                            stepmode="backward",
                        ),
                        dict(step="all"),
                    ]
                )
            ),
            rangeslider=dict(visible=True),
            type="date",
        )
    )

    fig.show()

In [ ]:
for intervenant_id in temp["Caregiver_ID"].unique():
    plot_agenda_for_intervenant(intervenant_id)

In [ ]:
# Loading and merging commute data
commute_file_paths = [
    "../data/commute_bicycling_clients.csv",
    "../data/commute_driving_clients.csv",
    "../data/commute_bicycling_care_clients.csv",
    "../data/commute_bicycling_clients_care.csv",
    "../data/commute_driving_care_clients.csv",
    "../data/commute_driving_clients_care.csv",
]


commute_dataframes = [pd.read_csv(file) for file in commute_file_paths]


for df in commute_dataframes:
    if df.columns[0] not in ["pair"]:  # standardizing column names
        df.rename(columns={df.columns[0]: "pair"}, inplace=True)

commute_data_df = pd.concat(commute_dataframes, ignore_index=True)

In [ ]:
commute_data_df["commute_minutes"] = commute_data_df["commute_seconds"] / 60

In [ ]:
caregivers["Commute Method"] = "driving"  # map commute method
jan24_df = temp.merge(
    caregivers[["ID Intervenant", "Commute Method"]],
    right_on="ID Intervenant",
    left_on="Caregiver_ID",
    how="left",
)  # merge with agenda data

jan24_df["Start DateTime"] = pd.to_datetime(
    jan24_df["Date"].astype(str) + " " + jan24_df["Heure de début"].astype(str)
)
jan24_df["End DateTime"] = pd.to_datetime(
    jan24_df["Date"].astype(str) + " " + jan24_df["Heure de fin"].astype(str)
)

commute_data_df[["source", "destination"]] = commute_data_df[
    "pair"
].str.extract(r"\((\d+), (\d+)\)")
commute_data_df.drop(columns="pair", inplace=True)
commute_data_df.set_index(
    ["source", "destination", "commute_method"], inplace=True
)

# Function to compute commute and wait times
def compute_commute_and_wait_times(df, commute_data_df):
    df["Wait Time"] = 0
    df["Commute Time"] = 0

    for intervenant_id in df["ID Intervenant"].unique():
        intervenant_data = df[df["ID Intervenant"] == intervenant_id]

        for date in intervenant_data["Date"].unique():
            daily_data = intervenant_data[
                intervenant_data["Date"] == date
            ].sort_values(by="Start DateTime")
            prev_end_time = None
            prev_client_id = (
                intervenant_id  # intervenant ID for the first trip
            )

            for index, row in daily_data.iterrows():
                source_id = str(prev_client_id)
                destination_id = str(row["ID Client"])
                commute_method = row["Commute Method"]

                try:
                    commute_time = commute_data_df.loc[
                        (source_id, destination_id, commute_method),
                        "commute_minutes",
                    ]
                except KeyError:
                    commute_time = 0
                # Calculate wait time
                if prev_end_time is not None:
                    wait_time = (
                        row["Start DateTime"] - prev_end_time
                    ).total_seconds() / 60
                else:
                    wait_time = 0

                # Update the dataframe
                df.loc[index, "Wait Time"] = wait_time
                df.loc[index, "Commute Time"] = commute_time

                # Update previous end time and client ID
                prev_end_time = row["End DateTime"]
                prev_client_id = row[
                    "ID Client"
                ]  # Update to client ID for subsequent trips

    return df

In [ ]:
def plot_agenda_for_intervenant(intervenant_id, jan24_df, commute_data_df):
    intervenant_agenda = jan24_df[jan24_df["ID Intervenant"] == intervenant_id]
    intervenant_agenda_sorted = intervenant_agenda.sort_values(
        by=["Date", "Heure de début"]
    )

    df_timeline = intervenant_agenda_sorted.copy()
    df_timeline["Start"] = pd.to_datetime(df_timeline["Start DateTime"])
    df_timeline["Finish"] = pd.to_datetime(df_timeline["End DateTime"])
    df_timeline["Task"] = df_timeline["Prestation"]
    df_timeline["Resource"] = df_timeline["ID Intervenant"].astype(str)
    df_timeline["ID Client"] = df_timeline["ID Client"].astype(str)

    # Compute wait and commute times
    df_timeline = compute_commute_and_wait_times(df_timeline, commute_data_df)

    fig = px.timeline(
        df_timeline,
        x_start="Start",
        x_end="Finish",
        y="Task",
        hover_data=["ID Client", "Wait Time", "Commute Time"],
    )
    fig.update_yaxes(autorange="reversed")
    fig.update_layout(title=f"Agenda for Intervenant ID: {intervenant_id}")
    fig.update_layout(
        xaxis=dict(
            rangeselector=dict(
                buttons=list(
                    [
                        dict(
                            count=1,
                            label="1D",
                            step="day",
                            stepmode="backward",
                        ),
                        dict(
                            count=7,
                            label="1W",
                            step="day",
                            stepmode="backward",
                        ),
                        dict(step="all"),
                    ]
                )
            ),
            rangeslider=dict(visible=True),
            type="date",
        )
    )

    fig.show()

    return df_timeline

In [ ]:
for intervenant_id in jan24_df["ID Intervenant"].unique():
    plot_agenda_for_intervenant(intervenant_id, jan24_df, commute_data_df)